In [4]:
from distance_map import load_distance_map
import cpmpy as cp
import warnings
warnings.filterwarnings("ignore")
import pulp

# Constraint Programming TSP

In [258]:
dc = 800
dist_map = load_distance_map(dc=dc)

In [259]:
tasks = [["155", "35"], ["25", "30"], ["55", "75"], ["155", "32"], ["10", "631"]]

In [260]:
num_tasks = len(tasks)

In [248]:
locns = set()
for task in tasks:
    locns.add(task[0])
    locns.add(task[1])
locns = sorted(list(locns))
locns

['10', '155', '25', '30', '32', '35', '55', '631', '75']

In [249]:
# Define decision variables (task order)
order = cp.intvar(0, num_tasks-1, shape=num_tasks, name="order")

In [250]:
# Calculate total distance
dist_map = load_distance_map(dc=dc)

def get_total_distance(order):
    distance = 0
    # for initial condition
    if order[0].value() is None:  
        distance += sum(dist_map.get((tasks[i][0], tasks[i][1]), 50) for i in range(len(order)))
        distance += sum(dist_map.get((tasks[i][1], tasks[i+1][0]), 50) for i in range(len(order)-1))
    else:
    # if orders has values
        distance += sum(dist_map.get((tasks[order[i].value()][0], tasks[order[i].value()][1]), 50) for i in range(len(order)))
        distance += sum(dist_map.get((tasks[order[i].value()][1], tasks[order[i+1].value()][0]), 50) for i in range(len(order)-1))
    return distance

In [251]:
print(get_total_distance(order))

765


In [252]:
# Create model
# Define the objective function to minimize total distance

m = cp.Model(cp.AllDifferent(order), minimize=get_total_distance(order))


In [253]:
# Solve model
solution = m.solve()
print("Status:", m.status())

Status: ExitStatus.OPTIMAL (0.0089841 seconds)


In [254]:
if solution:
    print(m.objective_value) 
    task_order = [[] for _ in range(num_tasks)]
    for i in range(len(order)):
        task_order[i] = tasks[order[i].value()]
    # task_order = [tasks[t.value()] for t in order]

<bound method Model.objective_value of Constraints:
    alldifferent(order[0],order[1],order[2],order[3],order[4])
Objective: minimize 765>


In [255]:
task_order

[['25', '30'], ['55', '75'], ['155', '32'], ['10', '631'], ['155', '35']]

In [256]:
print(get_total_distance(order))

798


# Try 2 TSP

In [296]:
from ortools.sat.python import cp_model

In [322]:
def get_distance(task1, task2):
  """Fetches distance between two tasks from the distance map."""
  return dist_map.get(((task1[1], task2[0])), 50)  # Default 50 if not found

In [323]:
model = cp_model.CpModel()
num_tasks = len(tasks)
order = [model.new_int_var(0, num_tasks - 1, f"order_{i}") for i in range(num_tasks)]

In [324]:
model.add_all_different(order)

In [329]:
model.minimize(sum([order[i] for i in range(num_tasks)]))

In [334]:
sum([tasks[order[i]] for i in range(num_tasks)])

AttributeError: 'IntVar' object has no attribute 'value'

In [330]:
solver = cp_model.CpSolver()
status = solver.solve(model)

In [331]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"Maximum of objective function: {solver.objective_value}\n")
    for i in range(len(order)):
        print(f"order_{i} = {solver.value(order[i])}")

Maximum of objective function: 10.0

order_0 = 4
order_1 = 3
order_2 = 2
order_3 = 1
order_4 = 0


# TSP OR-Tools

In [337]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [415]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            if manager.IndexToNode(index) != 0:
                plan_output += f" {data["locns"][manager.IndexToNode(index)-1]} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        # plan_output += f"{data["locns"][manager.IndexToNode(index)-1]}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        total_distance += route_distance
    print(f"Total Distance of all routes: {total_distance}m")

In [414]:
"""Single Vehicle Pickup Delivery Problem (PDP)."""


def create_data_model():
    """Stores the data for the problem."""
    dc = 800
    data = {}
    data["pickups_deliveries"] = [["155", "35"], ["25", "30"], ["155_1", "32"]]
    data["locns"] = sorted(list(set([_[0] for _ in data["pickups_deliveries"]] + [_[1] for _ in data["pickups_deliveries"]])))

    # Calculate distance matrix (depot is 0 with no distance)
    dist_map = load_distance_map(dc=dc)

    # Create distance matrix with fake depot (index 0)
    data["distance_matrix"] = [[0 for _ in range(len(data["locns"]) + 1)] for _ in range(len(data["locns"]) + 1)]
    for i in range(len(data["locns"])):
        locn_i = data["locns"][i]
        for j in range(len(data["locns"])):
            locn_j = data["locns"][j]
            if locn_i != locn_j:
                data["distance_matrix"][i+1][j+1] = dist_map.get((locn_i, locn_j), 50)
    # print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in data["distance_matrix"]]))
    
    data["num_vehicles"] = 1
    data["capacity"] = [1]
    data["depot"] = 0
    return data

In [468]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()
    locns = data["locns"]
    from_locns = set([_[0] for _ in data["pickups_deliveries"]])
    to_locns = set([_[1] for _ in data["pickups_deliveries"]])

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        10000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # # Capacity constraint
    # def demand_callback(index):
    #     """Returns the demand of the node."""
    #     # Convert from routing variable Index to demands NodeIndex.
    #     node = manager.IndexToNode(index)
    #     if locns[node+1] in from_locns:
    #         return 1
    #     elif locns[node+1] in to_locns:
    #         return -1
    #     else:
    #         return 0
    
    # demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    # routing.AddDimensionWithVehicleCapacity(
    #     demand_callback_index,
    #     0,  # null capacity slack
    #     data["capacity"],  # vehicle maximum capacities
    #     True,  # start cumul to zero
    #     "Capacity",
    # )

    # Define Transportation Requests.
    for request in data["pickups_deliveries"]:
        pickup_index = manager.NodeToIndex(locns.index(request[0])+1)
        delivery_index = manager.NodeToIndex(locns.index(request[1])+1)
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index)
        )
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index) + data["distance_matrix"][locns.index(request[0])+1][locns.index(request[1])+1]
            == distance_dimension.CumulVar(delivery_index) 
        )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found.")

In [469]:
main()
# [["155", "35"], ["25", "30"], ["155_1", "32"]]

No solution found.


# TSP Package

In [1]:
dc = 800

In [2]:
# Make tasks distinct
tasks = [["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"]]

In [5]:
dist_map = load_distance_map(dc=dc)

In [65]:
# Distance matrix from end_loc (row) -> start loc (col)
import numpy as np


distance_matrix = [[0 for _ in range(len(tasks))] for _ in range(len(tasks))]
for i in range(len(tasks)):
    for j in range(len(tasks)):
        if i == j:
            distance_matrix[i][j] = 0
        elif tasks[i][1] == tasks[j][0]:
            distance_matrix[i][j] = 0
        else:
            # from task i to task j
            distance_matrix[i][j] = dist_map[(tasks[i][1], tasks[j][0])]
        
distance_matrix = np.array(distance_matrix)
# Add row of 0s (fake depot)
zero_row = np.zeros(distance_matrix.shape[1])
distance_matrix = np.vstack([zero_row, distance_matrix])
# Add col of 0s (fake depot)
zero_col = np.zeros((distance_matrix.shape[0], 1))
distance_matrix = np.hstack([zero_col, distance_matrix])

0	14	201	154	154	52	26	14	201	154	154	52	26	14	201	154	154	52
46	0	165	118	118	34	46	48	165	118	118	34	46	48	165	118	118	34
70	54	0	110	110	16	70	54	157	110	110	16	70	54	157	110	110	16
24	26	187	0	140	56	24	26	187	140	140	56	24	26	187	140	140	56
28	0	199	152	0	50	28	0	199	152	152	50	28	0	199	152	152	50
193	195	6	57	57	0	193	195	6	57	57	157	193	195	6	57	57	157
26	14	201	154	154	52	0	14	201	154	154	52	26	14	201	154	154	52
46	48	165	118	118	34	46	0	165	118	118	34	46	48	165	118	118	34
70	54	157	110	110	16	70	54	0	110	110	16	70	54	157	110	110	16
24	26	187	140	140	56	24	26	187	0	140	56	24	26	187	140	140	56
28	0	199	152	152	50	28	0	199	152	0	50	28	0	199	152	152	50
193	195	6	57	57	157	193	195	6	57	57	0	193	195	6	57	57	157
26	14	201	154	154	52	26	14	201	154	154	52	0	14	201	154	154	52
46	48	165	118	118	34	46	48	165	118	118	34	46	0	165	118	118	34
70	54	157	110	110	16	70	54	157	110	110	16	70	54	0	110	110	16
24	26	187	140	140	56	24	26	187	140	140	56	24	26	187	0	140	56
28	0	199	152	152	50	28	0	199	1

In [66]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in distance_matrix]))


0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
0.0	0.0	14.0	201.0	154.0	154.0	52.0	26.0	14.0	201.0	154.0	154.0	52.0	26.0	14.0	201.0	154.0	154.0	52.0
0.0	46.0	0.0	165.0	118.0	118.0	34.0	46.0	48.0	165.0	118.0	118.0	34.0	46.0	48.0	165.0	118.0	118.0	34.0
0.0	70.0	54.0	0.0	110.0	110.0	16.0	70.0	54.0	157.0	110.0	110.0	16.0	70.0	54.0	157.0	110.0	110.0	16.0
0.0	24.0	26.0	187.0	0.0	140.0	56.0	24.0	26.0	187.0	140.0	140.0	56.0	24.0	26.0	187.0	140.0	140.0	56.0
0.0	28.0	0.0	199.0	152.0	0.0	50.0	28.0	0.0	199.0	152.0	152.0	50.0	28.0	0.0	199.0	152.0	152.0	50.0
0.0	193.0	195.0	6.0	57.0	57.0	0.0	193.0	195.0	6.0	57.0	57.0	157.0	193.0	195.0	6.0	57.0	57.0	157.0
0.0	26.0	14.0	201.0	154.0	154.0	52.0	0.0	14.0	201.0	154.0	154.0	52.0	26.0	14.0	201.0	154.0	154.0	52.0
0.0	46.0	48.0	165.0	118.0	118.0	34.0	46.0	0.0	165.0	118.0	118.0	34.0	46.0	48.0	165.0	118.0	118.0	34.0
0.0	70.0	54.0	157.0	110.0	110.0	16.0	70.0	54.0	0.0	110.0	110.0	16.0	70.0	54.0	157.0	110.0	110.0	16.0
0.0	24.0	26.0	187

In [52]:
from python_tsp.heuristics import solve_tsp_lin_kernighan, solve_tsp_record_to_record, solve_tsp_local_search
import numpy as np
distance_matrix_arr = np.array(distance_matrix)
distance_matrix_arr[:, 0] = 0
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in distance_matrix_arr]))

0	14	201	154	154	52	26	14	201	154	154	52	26	14	201	154	154	52
0	0	165	118	118	34	46	48	165	118	118	34	46	48	165	118	118	34
0	54	0	110	110	16	70	54	157	110	110	16	70	54	157	110	110	16
0	26	187	0	140	56	24	26	187	140	140	56	24	26	187	140	140	56
0	0	199	152	0	50	28	0	199	152	152	50	28	0	199	152	152	50
0	195	6	57	57	0	193	195	6	57	57	157	193	195	6	57	57	157
0	14	201	154	154	52	0	14	201	154	154	52	26	14	201	154	154	52
0	48	165	118	118	34	46	0	165	118	118	34	46	48	165	118	118	34
0	54	157	110	110	16	70	54	0	110	110	16	70	54	157	110	110	16
0	26	187	140	140	56	24	26	187	0	140	56	24	26	187	140	140	56
0	0	199	152	152	50	28	0	199	152	0	50	28	0	199	152	152	50
0	195	6	57	57	157	193	195	6	57	57	0	193	195	6	57	57	157
0	14	201	154	154	52	26	14	201	154	154	52	0	14	201	154	154	52
0	48	165	118	118	34	46	48	165	118	118	34	46	0	165	118	118	34
0	54	157	110	110	16	70	54	157	110	110	16	70	54	0	110	110	16
0	26	187	140	140	56	24	26	187	140	140	56	24	26	187	0	140	56
0	0	199	152	152	50	28	0	199	152	152	50	28	0	199

In [63]:
permutation, distance = solve_tsp_record_to_record(distance_matrix_arr)

In [59]:
verify_distance = 0
for i in range(len(permutation)-1):
    index = permutation[i]
    next_index = permutation[i+1]
    curr_task = tasks[index]
    next_task = tasks[next_index]
    if curr_task[1] != next_task[0]:
        verify_distance += dist_map[(curr_task[1], next_task[0])]
verify_distance

908

# VRP

In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
from distance_map import load_distance_map
dc = 800

In [3]:
def print_solution(data, manager, routing, solution):
    """Prints VRP (OR Tools) solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")

In [26]:
tasks = [["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"]]
distance_map = load_distance_map(dc=dc)
distance_matrix = [[0 for _ in range(len(tasks))] for _ in range(len(tasks))]
for i in range(len(tasks)):
    for j in range(len(tasks)):
        if i == j:
            distance_matrix[i][j] = 0
        elif tasks[i][1] == tasks[j][0]:
            distance_matrix[i][j] = 0
        else:
            # from task i to task j
            distance_matrix[i][j] = distance_map[
                (tasks[i][1], tasks[j][0])
            ]
distance_matrix = np.array(distance_matrix)
# Add row of 0s (fake depot)
zero_row = np.zeros(distance_matrix.shape[1])
distance_matrix = np.vstack([zero_row, distance_matrix])
# Add col of 0s (fake depot)
zero_col = np.zeros((distance_matrix.shape[0], 1))
distance_matrix = np.hstack([zero_col, distance_matrix])
distance_matrix = distance_matrix.tolist()
for i in range(len(distance_matrix)):
    for j in range(len(distance_matrix)):
        distance_matrix[i][j] = int(distance_matrix[i][j])
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in distance_matrix]))

0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0
0	0	14	201	154	154	52	26	14	201	154	154	52	26	14	201	154	154	52	26	14	201	154	154	52	26	14	201	154	154	52	26	14	201	154	154	52	26	14	201	154	154	52	26	14	201	154	154	52	26	14	201	154	154	52
0	46	0	165	118	118	34	46	48	165	118	118	34	46	48	165	118	118	34	46	48	165	118	118	34	46	48	165	118	118	34	46	48	165	118	118	34	46	48	165	118	118	34	46	48	165	118	118	34	46	48	165	118	118	34
0	70	54	0	110	110	16	70	54	157	110	110	16	70	54	157	110	110	16	70	54	157	110	110	16	70	54	157	110	110	16	70	54	157	110	110	16	70	54	157	110	110	16	70	54	157	110	110	16	70	54	157	110	110	16
0	24	26	187	0	140	56	24	26	187	140	140	56	24	26	187	140	140	56	24	26	187	140	140	56	24	26	187	140	140	56	24	26	187	140	140	56	24	26	187	140	140	56	24	26	187	140	140	56	24	26	187	140	140	56
0	28	0	199	152	0	50	28	0	199	152	152	50	28	0	199	152	152	50	28	0	199	152	152	50	28	0	199	152	152	50	28	0	199	152	152	50	28	0	199	152

In [27]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distance_matrix
    data["num_vehicles"] = 18
    data["depot"] = 0
    return data


In [28]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(
            len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
        )
routing = pywrapcp.RoutingModel(manager)

In [29]:
# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index) 
    return data["distance_matrix"][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [30]:
data["distance_matrix"]

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  14,
  201,
  154,
  154,
  52,
  26,
  14,
  201,
  154,
  154,
  52,
  26,
  14,
  201,
  154,
  154,
  52,
  26,
  14,
  201,
  154,
  154,
  52,
  26,
  14,
  201,
  154,
  154,
  52,
  26,
  14,
  201,
  154,
  154,
  52,
  26,
  14,
  201,
  154,
  154,
  52,
  26,
  14,
  201,
  154,
  154,
  52,
  26,
  14,
  201,
  154,
  154,
  52],
 [0,
  46,
  0,
  165,
  118,
  118,
  34,
  46,
  48,
  165,
  118,
  118,
  34,
  46,
  48,
  165,
  118,
  118,
  34,
  46,
  48,
  165,
  118,
  118,
  34,
  46,
  48,
  165,
  118,
  118,
  34,
  46,
  48,
  165,
  118,
  118,
  34,
  46,
  48,
  165,
  118,
  118,
  34,
  46,
  48,
  165,
  118,
  118,
  34,
  46,
  48,
  165,
  118,
  118,
  34],
 [0,
  70,
 

In [31]:
 # Add Distance constraint.
dimension_name = "Distance"
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    1000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name,
)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

In [32]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

In [33]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [34]:
# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)
else:
    print("No solution found !")

Objective: 9654
Route for vehicle 0:
 0 ->  33 ->  36 ->  39 ->  42 ->  45 ->  48 ->  51 ->  54 ->  3 -> 0
Distance of the route: 88m

Route for vehicle 1:
 0 ->  23 ->  26 -> 0
Distance of the route: 0m

Route for vehicle 2:
 0 ->  28 ->  49 ->  1 -> 0
Distance of the route: 50m

Route for vehicle 3:
 0 ->  22 ->  19 -> 0
Distance of the route: 24m

Route for vehicle 4:
 0 ->  5 ->  44 ->  6 ->  34 -> 0
Distance of the route: 91m

Route for vehicle 5:
 0 ->  15 ->  30 ->  41 ->  8 -> 0
Distance of the route: 73m

Route for vehicle 6:
 0 ->  46 ->  43 -> 0
Distance of the route: 24m

Route for vehicle 7:
 0 ->  40 ->  37 -> 0
Distance of the route: 24m

Route for vehicle 8:
 0 ->  53 ->  2 -> 0
Distance of the route: 0m

Route for vehicle 9:
 0 ->  10 ->  13 -> 0
Distance of the route: 24m

Route for vehicle 10:
 0 ->  16 ->  7 -> 0
Distance of the route: 24m

Route for vehicle 11:
 0 ->  17 ->  38 ->  24 ->  9 ->  12 ->  27 ->  18 ->  21 -> 0
Distance of the route: 84m

Route for vehi

# VRP 2

In [4]:
import pyvrp
from distance_map import load_distance_map
import numpy as np
from pyvrp import Model
dc = 800

In [5]:
tasks = [["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"], ["25", "30"], ["32", "55"], ["IBNP", "72"], ["155", "35"], ["155", "32"], ["68", "631"]]
distance_map = load_distance_map(dc=dc)
distance_matrix = [[0 for _ in range(len(tasks))] for _ in range(len(tasks))]
for i in range(len(tasks)):
    for j in range(len(tasks)):
        if i == j:
            distance_matrix[i][j] = 0
        elif tasks[i][1] == tasks[j][0]:
            distance_matrix[i][j] = 0
        else:
            # from task i to task j
            distance_matrix[i][j] = distance_map[
                (tasks[i][1], tasks[j][0])
            ]
distance_matrix = np.array(distance_matrix)
# Add row of 0s (fake depot)
zero_row = np.zeros(distance_matrix.shape[1])
distance_matrix = np.vstack([zero_row, distance_matrix])
# Add col of 0s (fake depot)
zero_col = np.zeros((distance_matrix.shape[0], 1))
distance_matrix = np.hstack([zero_col, distance_matrix])
distance_matrix = distance_matrix.tolist()
for i in range(len(distance_matrix)):
    for j in range(len(distance_matrix)):
        distance_matrix[i][j] = int(distance_matrix[i][j])
# print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in distance_matrix]))


In [6]:
# Instantiate model and create depot/clients
m = Model()
num_vehicles = 4
m.add_vehicle_type(num_vehicles, capacity=int(len(tasks) / num_vehicles) + 2)
depot = m.add_depot(x=0, y=0)
clients = [
    m.add_client(x=0, y=0, delivery=1)
    for idx in range(1, len(distance_matrix))
]

# Add edges based on distance matrix
locations = [depot] + clients
for i in range(len(locations)):
    for j in range(len(locations)):
        distance = distance_matrix[i][j]
        m.add_edge(locations[i], locations[j], distance=distance)

In [7]:
from pyvrp.stop import MaxRuntime

res = m.solve(stop=MaxRuntime(1), display=True)  # one second

PyVRP v0.8.2

Solving an instance with:
    1 depot
    54 clients
    4 vehicles (1 vehicle type)

                  |       Feasible        |      Infeasible
    Iters    Time |   #      Avg     Best |   #      Avg     Best

Search terminated in 1.00s after 199 iterations.
Best-found solution has cost 2194.



In [8]:
print(res)

Solution results
    # routes: 4
   # clients: 54
   objective: 2194.00
# iterations: 199
    run-time: 1.00 seconds

Routes
------
Route #1: 34 22 23 38 53 20 35 26 4 37 49 
Route #2: 16 31 36 27 24 45 48 51 6 15 54 21 28 13 
Route #3: 41 14 17 44 5 50 11 32 29 2 47 8 40 25 
Route #4: 52 10 19 43 42 33 18 9 12 3 30 39 46 1 7 



In [10]:
routes = []
for x in res.best.routes():
    route = str(x)
    routes.append(list(map(int, route.split())))

In [11]:
routes

[[34, 22, 23, 38, 53, 20, 35, 26, 4, 37, 49],
 [16, 31, 36, 27, 24, 45, 48, 51, 6, 15, 54, 21, 28, 13],
 [41, 14, 17, 44, 5, 50, 11, 32, 29, 2, 47, 8, 40, 25],
 [52, 10, 19, 43, 42, 33, 18, 9, 12, 3, 30, 39, 46, 1, 7]]

In [12]:
route = routes[0]
for i in range(len(route)):
    idx = route[i] - 1
    print(tasks[idx])


['155', '35']
['155', '35']
['155', '32']
['32', '55']
['155', '32']
['32', '55']
['155', '32']
['32', '55']
['155', '35']
['25', '30']
['25', '30']


In [1]:
import datetime
from data_load import load_aggregate_data
from distance_map import load_distance_map
from models import VRP
dc = 800
dist_map = load_distance_map(dc=dc)
df = load_aggregate_data(dc=dc, date=datetime.datetime(2024, 5, 25)).head(1000)
routing = VRP(dist_map, df, 10)
print(routing.assign_tasks())


PyVRP v0.8.2

Solving an instance with:
    1 depot
    1000 clients
    10 vehicles (1 vehicle type)

                  |       Feasible        |      Infeasible
    Iters    Time |   #      Avg     Best |   #      Avg     Best

Search terminated in 5.22s after 16 iterations.
Best-found solution has cost 38109.

{0: [['110', '117'], ['110', '125'], ['124', '28'], ['27', '103'], ['106', '103'], ['104', '91'], ['100', '54'], ['104', '153'], ['110', '142'], ['134', '142'], ['110', 'IBNP'], ['168', '621'], ['622', '57'], ['27', '159'], ['132', '59'], ['104', '56'], ['104', '148'], ['110', '142'], ['128', '63'], ['106', '129'], ['110', '79'], ['100', '49'], ['37', '147'], ['110', '145'], ['128', '146'], ['122', '157'], ['122', '95'], ['108', '146'], ['124', '113'], ['110', '143'], ['128', '161'], ['122', '105'], ['106', '117'], ['118', '95'], ['100', '119'], ['110', '119'], ['118', '107'], ['108', '154'], ['128', '54'], ['104', '85'], ['104', '85'], ['100', '55'], ['33', '621'], ['622', '1

In [9]:
from data_load import load_aggregate_data
import datetime
df = load_aggregate_data(dc=800, date=datetime.datetime(2024, 6, 12))

In [10]:
df

,id,from_time,from_locn,to_time,to_locn,user,time_taken
288191,UW14064071,2024-06-12 00:00:02,27,2024-06-12 00:02:30,113,90,148.0
288192,UW14064070,2024-06-12 00:00:04,27,2024-06-12 00:01:24,155,90,80.0
288193,UT13673386,2024-06-12 00:00:11,108,2024-06-12 00:01:16,91,154,65.0
288194,UT13673411,2024-06-12 00:00:12,108,2024-06-12 00:00:23,91,24,11.0
288195,UT13673387,2024-06-12 00:00:13,108,2024-06-12 00:01:14,91,154,61.0
...,...,...,...,...,...,...,...
292689,UW14074422,2024-06-12 23:55:08,682,2024-06-12 23:58:16,153,37,188.0
292690,UT13813432,2024-06-12 23:55:33,622,2024-06-12 23:56:41,71,35,68.0
292691,UM14321548,2024-06-12 23:56:04,622,2024-06-12 23:57:18,53,106,74.0
292692,UW14073707,2024-06-12 23:57:29,622,2024-06-12 23:58:38,117,154,69.0


In [17]:
improvements = [2.7058823529411766, 2.4579560155239326, 2.8967254408060454, 3.0913978494623655, 2.9150822382762991, 1.8229166666666667, 2.085889570552147, 1.9296254256526675, 1.9767441860465116, 1.8181818181818181, 2.4213075060532687, 3.234501347708895, 2.0486555697823303, 1.7921146953405018, 2.288329519450801, 2.2360248447204967, 1.3924050632911393, 1.3681592039800996, 2.2339027595269383, 1.8137847642079807, 2.1300448430493275, 2.219482120838471, 2.137767220902613, 2.2929936305732483, 3.0065359477124183, 1.3513513513513513, 2.4390243902439024]
print(list(map(round, improvements)))

[3, 2, 3, 3, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 3, 1, 2]
